In [ ]:
from get_data import *
pid = 'a3d13b05-bf4d-427a-a2d5-2fe050d603ec'
eid = '03cf52f6-fba6-4743-a42e-dd1ac3072343' 
# channels = get_channels(eid, pid, mode= 'download')

In [ ]:
from firing_rate import *
z_score_right, z_score_left, times, depths, ids, acronyms, ch_indexs, coordinates = right_left_firingRates_onDepths(eid, pid, depth_lim= [10, 3850])

In [ ]:
time = np.arange(-0.4, 1  , 0.1)

In [ ]:
behavior = get_behavior(eid, mode='download')
right_onset = behavior[behavior['contrastRight'] == 1]['stimOn_times']
left_onset = behavior[behavior['contrastLeft'] == 1]['stimOn_times']

# the index of trials
indx_right = list(right_onset.index)
indx_left = list(left_onset.index)
trial_indx = np.concatenate((indx_right, indx_left))

# the distance to the last change 
change_indices = behavior['probabilityLeft'].ne(behavior['probabilityLeft'].shift()).to_numpy().nonzero()[0]
distance_to_change = np.array([0 if i in change_indices else i - change_indices[change_indices < i][-1] for i in range(len(behavior))])


probs_left = behavior['probabilityLeft'][trial_indx].reset_index(drop=True)

In [13]:
import numpy as np

def bin_spike_dataa(spike_times, spike_depths, time_bin, depth_bin, time_range, depth_range=[10, 3850]):
    # Determine the time and depth limits
    time_min, time_max = time_range
    depth_min, depth_max = depth_range
    
    def _get_scale_and_indices(values, bin_size, limits):
        # Calculate the scale (edges) and indices for binning
        scale = np.arange(limits[0], limits[1] + bin_size, bin_size)
        indices = np.clip((np.floor((values - limits[0]) / bin_size)).astype(int), 0, len(scale) - 2)
        return scale, indices
    
    # Get time and depth scales and indices
    time_scale, time_indices = _get_scale_and_indices(spike_times, time_bin, (time_min, time_max))
    depth_scale, depth_indices = _get_scale_and_indices(spike_depths, depth_bin, (depth_min, depth_max))
    
    # Create 2D histogram array by aggregating with bincount on indices
    binned_spikes = np.zeros((len(depth_scale) - 1, len(time_scale) - 1))
    ind2d = np.ravel_multi_index((depth_indices, time_indices), dims=binned_spikes.shape)
    binned_counts = np.bincount(ind2d, minlength=binned_spikes.size)
    binned_spikes.flat = binned_counts  # reshape to 2D array
    
    # Calculate centers of each bin to match binned_spikes dimensions
    time_centers = (time_scale[:-1] + time_scale[1:]) / 2
    depth_centers = (depth_scale[:-1] + depth_scale[1:]) / 2
    
    return binned_spikes, time_centers, depth_centers


In [21]:
pid = 'e4ce2e94-6fb9-4afe-acbf-6f5a3498602e'
eid = '6ab9d98c-b1e9-4574-b8fe-b9eec88097e0' 
modee = 'download'
import numpy as np
from get_data import *
######################
# Load data
########################
# Load behavior
behavior = get_behavior(eid, modee=modee)
right_onset = behavior[behavior['contrastRight'] == 1]['stimOn_times']
left_onset = behavior[behavior['contrastLeft'] == 1]['stimOn_times']
stim_events = {'right': right_onset, 'left': left_onset}

# the index of trials
indx_right = list(right_onset.index)
indx_left = list(left_onset.index)
trial_indx = np.concatenate((indx_right, indx_left))

# the distance to the last change 
change_indices = behavior['probabilityLeft'].ne(behavior['probabilityLeft'].shift()).to_numpy().nonzero()[0]
distance_to_change = np.array([0 if i in change_indices else i - change_indices[change_indices < i][-1] for i in range(len(behavior))])
distance_to_change = distance_to_change[trial_indx]
# the probability of left
probs_left = behavior['probabilityLeft'][trial_indx].reset_index(drop=True)

# load channel data 
channels = get_channels(eid, pid,  modee=modee) # download the data from the IBL database or 'load' data from the local directory
# load spikes
spikes_datasets = get_spikes(pid, modee=modee)
spikes = spikes_datasets['spikes']
spike_times = spikes['times']
spike_depths = spikes['depths']
kp_idx = np.where(~np.isnan(spike_depths))[0] # Remove any nan depths
spike_times = spike_times[kp_idx]
spike_depths = spike_depths[kp_idx]
######################
# compute firing rate
    ########################
pre_stim = 0.4
post_stim = 1
t_bin = 0.1
d_bin = 20
depth_lim = [10, 3840]



In [22]:
from iblutil.numerical import bincount2D
# Generate time and depth arrays
pre_stim = 0.4
post_stim = 1
t_bin = 0.1
d_bin = 20
depth_lim = [10, 3840]
z_scores = {stim_type: [] for stim_type in stim_events.keys()}

for stim_type, stim_times in stim_events.items():
    stim_times = stim_times[~np.isnan(stim_times)]
    for stim_on_time in stim_times:
        interval = [stim_on_time - pre_stim, stim_on_time + post_stim]
        idx = np.where((spike_times > interval[0]) & (spike_times < interval[1]))[0]
        spike_times_i = spike_times[idx]
        spike_depths_i = spike_depths[idx]

        # Create binned array
        binned_array, tim, depths = bincount2D(spike_times_i, spike_depths_i, xbin= t_bin, ybin= d_bin, xlim=interval, ylim=depth_lim)
        
        # Compute baseline mean and std
        baseline_mean = np.mean(binned_array[:, :int(pre_stim/t_bin)], axis=1)
        baseline_std = np.std(binned_array[:, :int(pre_stim/t_bin)], axis=1)
        baseline_std[baseline_std == 0] = 1  # Avoid division by zero

        # Compute Z-score of firing rates
        z_score_firing_rate = (binned_array - baseline_mean[:, np.newaxis]) / baseline_std[:, np.newaxis]
  
        # Append to the respective list
        z_scores[stim_type].append(z_score_firing_rate)

z_scores = {stim_type: np.array(z_score_list) for stim_type, z_score_list in z_scores.items()}

time_centers = tim - stim_on_time
depths_centers = depths + d_bin/2
    


In [29]:
z_score_right = z_scores['right']
z_score_left = z_scores['left']
######################
# Get depth information
ids = []
acronyms = []
true_depths = []
ch_indexs = []
cordinates = []
for depth in depths_centers:
    id = channels[channels['axial_um'] == depth]['atlas_id']
    acronym = channels[channels['axial_um'] == depth]['acronym']
    ch_index = channels[channels['axial_um'] == depth].index
    coordinate = channels[channels['axial_um'] == depth][['x', 'y', 'z']]
    if len(id) > 0:
        id = id.values[0]
        acro = acronym.values[0]
        ch_index = ch_index[0]
        coordinate = coordinate.values[0]
        ch_indexs.append(ch_index)
        acronyms.append(acro)
        ids.append(id)
        true_depths.append(depth)
        cordinates.append(coordinate)
ids = np.array(ids)
acronyms = np.array(acronyms)
ch_indexs = np.array(ch_indexs)
coordinates = np.array(cordinates)
trial_indx = np.array(trial_indx)
distance_to_change = np.array(distance_to_change)
probs_left = np.array(probs_left)

In [32]:
average_period = [0.1, 1]
time_indices = np.where((time_centers >= average_period[0]) & (time_centers <= average_period[1]))[0]

# Average firing rates over the defined time window
z_score_right_average = np.mean(z_score_right[:, :, time_indices], axis=2)
z_score_left_average = np.mean(z_score_left[:, :, time_indices], axis=2)

# Labels for right (1) and left (0) trials
labels_c1 = np.ones(z_score_right.shape[0])
labels_c2 = np.zeros(z_score_left.shape[0])
labels = np.concatenate((labels_c1, labels_c2))

# Stack firing rates data along trial dimension
firing_rates = np.concatenate((z_score_right_average, z_score_left_average), axis=0)

# Expand metadata to match firing rates data shape
x_coords = np.tile(coordinates[:, 0], firing_rates.shape[0])
y_coords = np.tile(coordinates[:, 1], firing_rates.shape[0])
z_coords = np.tile(coordinates[:, 2], firing_rates.shape[0])
acronyms_repeated = np.tile(acronyms, firing_rates.shape[0])
ch_indexs_repeated = np.tile(ch_indexs, firing_rates.shape[0])
labels_repeated = np.tile(labels, firing_rates.shape[1])
trial_indx_repeated = np.tile(trial_indx, firing_rates.shape[1])
distance_to_change_repeated = np.tile(distance_to_change, firing_rates.shape[1])
probs_left_repeated = np.tile(probs_left, firing_rates.shape[1])

# Repeat probe and experiment IDs to match data structure
probe_ids = np.repeat(pid, firing_rates.size)
experiment_ids = np.repeat(eid, firing_rates.size)


# print the shape of all the variables 
print(f'x_coords shape: {x_coords.shape}')
print(f'y_coords shape: {y_coords.shape}')



x_coords shape: (37248,)
y_coords shape: (37248,)


In [36]:
# Check shapes of original arrays before flattening/repeating
print("Original array shapes:")
print(f"firing_rates: {firing_rates.shape}")
print(f"labels_repeated: {labels_repeated.shape}")
print(f"trial_indx_repeated: {trial_indx_repeated.shape}")
print(f"distance_to_change_repeated: {distance_to_change_repeated.shape}")
print(f"probs_left_repeated: {probs_left_repeated.shape}")
print(f"x_coords: {x_coords.shape}")
print(f"y_coords: {y_coords.shape}")
print(f"z_coords: {z_coords.shape}")
print(f"ch_indexs_repeated: {ch_indexs_repeated.shape}")
print(f"acronyms_repeated: {len(acronyms_repeated)}")  # if it's a list
print(f"probe_ids: {len(probe_ids)}")  # if it's a list
print(f"experiment_ids: {len(experiment_ids)}")  # if it's a list


Original array shapes:
firing_rates: (194, 192)
labels_repeated: (37248,)
trial_indx_repeated: (37248,)
distance_to_change_repeated: (37248,)
probs_left_repeated: (37248,)
x_coords: (37248,)
y_coords: (37248,)
z_coords: (37248,)
ch_indexs_repeated: (37248,)
acronyms_repeated: 37248
probe_ids: 37248
experiment_ids: 37248


In [ ]:
z_scores_new = {}
for stim_type, z_score_list in z_scores.items():
    n = np.array(z_score_list)
    z_scores_new[stim_type] = z_score_list

In [ ]:
z_scores = {stim_type: [] for stim_type in stim_events.keys()}

for stim_type, stim_times in stim_events.items():
    stim_times = stim_times[~np.isnan(stim_times)]
    for stim_on_time in stim_times:
        interval = [stim_on_time - pre_stim, stim_on_time + post_stim]
        idx = np.where((spike_times > interval[0]) & (spike_times < interval[1]))[0]
        spike_times_i = spike_times[idx]
        spike_depths_i = spike_depths[idx]

        # Create binned array
        binned_array, t, depths = bin_spike_data(spike_times_i, spike_depths_i, t_bin, d_bin, time_range=interval, depth_range=depth_lim)

        # Compute baseline mean and std
        baseline_mean = np.mean(binned_array[:, :int(pre_stim/t_bin)], axis=1)
        baseline_std = np.std(binned_array[:, :int(pre_stim/t_bin)], axis=1)
        baseline_std[baseline_std == 0] = 1  # Avoid division by zero

        # Compute Z-score of firing rates
        z_score_firing_rate = (binned_array - baseline_mean[:, np.newaxis]) / baseline_std[:, np.newaxis]
        print(z_score_firing_rate.shape)
        # Append to the respective list
        z_scores[stim_type].append(z_score_firing_rate)